In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
df = pd.read_csv("../data/flags/flags.csv", header=None)

#y = np.array(df[0])

#x = np.array([df[1][i]+","+df[2][i]+","+df[3][i] for i in range(0,len(y))])

x = [df[1][i]+","+df[2][i] for i in range(0,14)]

y = list(df[3])

x = np.array(x + list(df[1]))

y = np.array(y + list(df[2]))

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20)

tokenizer.fit_on_texts(y)


y = tokenizer.texts_to_sequences(y)
y = np.array([elem[0]-1 for elem in y])

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20, split=",")

# updates internal vocabulary based on the lyrics
tokenizer.fit_on_texts(x)

# transform each text in x to a sequence of tokens
x = tokenizer.texts_to_sequences(x)

x = np.array(pad_sequences(x, maxlen = 2))

print(x)

In [ ]:
num = len(y)

#idx = np.random.permutation(len(x))
#print(idx)
#x = x[idx]
#y = y[idx]

#x_train, x_test = x[:int(num*0.8)], x[int(num*0.8):]
#y_train, y_test = y[:int(num*0.8)], y[int(num*0.8):]

x = x.reshape((x.shape[0], x.shape[1], 1))
#x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
#x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y))

np.unique(y)

In [ ]:
print(n_classes)

In [ ]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


In [ ]:
input_shape = x.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x,
    y,
    #validation_split=0.2,
    epochs=10000,
    batch_size=32,
    callbacks=callbacks,
)

model.evaluate(x, y, verbose=1)